<a href="https://colab.research.google.com/github/Annie-Yeeun-Jang/dacon_timeseries/blob/main/final_model/%EC%8B%9C%EA%B3%84%EC%97%B4_1%ED%8C%80_private_inference_%EC%9D%BC%EC%82%AC%EC%9D%BC%EC%A1%B0.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

test 데이터 전처리 후 inference
=> ulsan_weather_06, dangjin_weather_06 최신 업데이트 버전으로 가져오기

* 이미지 한 장이 하루여야 함
* 72시간의 데이터를 갖고 73번째 시간대의 데이터를 예측하는것

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
from tqdm import tqdm
from google.colab import drive
from datetime import datetime, timedelta

import math
import time
import random

import lightgbm as lgb
from sklearn.model_selection import cross_val_score, train_test_split, KFold, GridSearchCV
from sklearn.metrics import mean_squared_error, mean_absolute_error
from sklearn.externals import joblib

drive.mount('./gdrive', force_remount=True)
plt.style.use("fivethirtyeight")
plt.rc('font', family='NanumBarunGothic')
plt.rc('axes', unicode_minus=False)
warnings.filterwarnings(action='ignore')

/usr/local/lib/python3.7/dist-packages/sklearn/externals/joblib/__init__.py:15: FutureWarning: sklearn.externals.joblib is deprecated in 0.21 and will be removed in 0.23. Please import this functionality directly from joblib, which can be installed with: pip install joblib. If this warning is raised when loading pickled models, you may need to re-serialize those models with scikit-learn 0.21+.
  warnings.warn(msg, category=FutureWarning)


Mounted at ./gdrive


In [2]:
dangjin_weather_06 = pd.read_csv("/content/gdrive/MyDrive/dacon_timeseries/final_model/private_data/sun/dangjin_weather_06.csv", encoding = 'utf-8')
ulsan_weather_06 = pd.read_csv("/content/gdrive/MyDrive/dacon_timeseries/final_model/private_data/sun/ulsan_weather_06.csv", encoding = 'utf-8')
#dangjin_sun = pd.read_csv("/content/gdrive/MyDrive/dacon_timeseries/final_model/private_data/sun/dangjin0702_0703_sunlight.csv", encoding = 'utf-8')
#ulsan_sun = pd.read_csv("/content/gdrive/MyDrive/dacon_timeseries/final_model/private_data/sun/ulsan0702_0703_sunlight.csv", encoding = 'utf-8')

In [3]:
dangjin_weather_06[-73:]

,Forecast_time,Humidity,Cloud,Temperature,WindDirection,WindSpeed,일조,일사,PrecipitationProb,PrecipitationForm,방위각,고도,적경,적위
142,2021-07-02 23:00:00,67.0,4.0,25.5,90.000000,1.900000,0.000000,0.000000,NaN,NaN,334 54 06.04,-25 40 56.98,06 48 26.65,22 58 04.32
143,2021-07-03 00:00:00,69.0,0.0,25.1,90.000000,1.500000,0.000000,0.000000,NaN,NaN,349 55 39.74,-29 18 54.54,06 48 37.09,22 57 52.21
144,2021-07-03 01:00:00,79.0,5.0,23.4,0.000000,0.200000,0.000000,0.000000,NaN,NaN,005 47 50.08,-29 46 00.84,06 48 47.53,22 57 40.20
145,2021-07-03 02:00:00,82.0,6.0,22.6,0.000000,0.400000,0.000000,0.000000,NaN,NaN,021 10 18.92,-26 58 05.26,06 48 57.97,22 57 28.28
146,2021-07-03 03:00:00,88.0,9.0,22.9,270.000000,1.100000,0.000000,0.000000,NaN,NaN,034 59 25.60,-21 19 18.80,06 49 08.40,22 57 16.45
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
210,2021-07-05 19:00:00,75.0,1.0,25.0,283.666667,2.633333,0.861254,0.182235,0.0,0.0,291 24 11.18,09 26 39.75,07 00 06.50,22 42 58.69
211,2021-07-05 20:00:00,80.0,1.0,24.0,280.333333,2.166667,0.304918,0.000000,0.0,0.0,300 05 59.96,-1 20 08.46,07 00 16.80,22 42 43.41
212,2021-07-05 21:00:00,85.0,1.0,23.0,277.000000,1.700000,0.000000,0.000000,0.0,0.0,309 55 10.20,-11 08 39.68,07 00 27.14,22 42 28.15
213,2021-07-05 22:00:00,85.0,1.0,23.0,277.000000,1.700000,0.000000,0.000000,0.0,0.0,321 20 21.13,-19 31 36.23,07 00 37.50,22 42 12.91


In [4]:
def interpolatee(dangjin_fcst):
  dangjin_fcst['Temperature'] = dangjin_fcst['Temperature'].interpolate()  #interpolate() 구간 따로 안정하고 column 단위로 통째로 적용하면 되는거 맞음
  #dangjin_fcst['PrecipitationForm'] = dangjin_fcst['PrecipitationForm'].interpolate()
  #dangjin_fcst['PrecipitationProb'] = dangjin_fcst['PrecipitationProb'].interpolate()
  dangjin_fcst['Humidity'] = dangjin_fcst['Humidity'].interpolate()
  dangjin_fcst['WindSpeed'] = dangjin_fcst['WindSpeed'].interpolate()
  dangjin_fcst['Cloud'] = dangjin_fcst['Cloud'].interpolate()
  #dangjin_fcst['Precipitation'] = dangjin_fcst['Precipitation'].interpolate()
  #dangjin_fcst['Snow'] = dangjin_fcst['Snow'].interpolate()
  return dangjin_fcst

In [5]:
dangjin_weather_06 = interpolatee(dangjin_weather_06)
ulsan_weather_06 = interpolatee(ulsan_weather_06)

In [6]:
#Wind Direction 항목 angle에서 방향표시로 바꿔주기
def angle_to_dir(x):
    if pd.isnull(x):
        return np.nan
    elif x >= 22.5 and x <67.5:
        return 'NE'
    elif x >= 67.5 and x < 112.5:
        return 'E'
    elif x >= 112.5 and x < 157.5:
        return 'SE'
    elif x >= 157.5 and x < 202.5:
        return 'S'
    elif x >= 202.5 and x < 247.5:
        return 'WS'
    elif x >= 247.5 and x < 292.5:
        return 'W'
    elif x >= 292.5 and x < 337.5:
        return 'NW'
    elif x >= 337.5 or x < 22.5:
        return 'N'

def angle_to_cos(x):
    return np.cos(np.pi/180*(x-90))
def angle_to_sin(x):
    return np.sin(np.pi/180*(x+90))


In [7]:
dangjin_weather_06['WindDirection_x'] = dangjin_weather_06['WindDirection'].apply(angle_to_cos)  
dangjin_weather_06['WindDirection_y'] = dangjin_weather_06['WindDirection'].apply(angle_to_sin)
dangjin_weather_06['Wind_x'] = dangjin_weather_06['WindDirection_x'] * dangjin_weather_06['WindSpeed']
dangjin_weather_06['Wind_y'] = dangjin_weather_06['WindDirection_y'] * dangjin_weather_06['WindSpeed']

ulsan_weather_06['WindDirection_x'] = ulsan_weather_06['WindDirection'].apply(angle_to_cos)
ulsan_weather_06['WindDirection_y'] = ulsan_weather_06['WindDirection'].apply(angle_to_sin)
ulsan_weather_06['Wind_x'] = ulsan_weather_06['WindDirection_x'] * ulsan_weather_06['WindSpeed']
ulsan_weather_06['Wind_y'] = ulsan_weather_06['WindDirection_y'] * ulsan_weather_06['WindSpeed']

dangjin_weather_06['WindDirection'] = dangjin_weather_06['WindDirection'].interpolate()
ulsan_weather_06['WindDirection'] = ulsan_weather_06['WindDirection'].interpolate()

#dangjin_weather_06['WindDirection_catg'] = dangjin_weather_06['WindDirection'].apply(angle_to_dir)
#ulsan_weather_06['WindDirection_catg'] = ulsan_weather_06['WindDirection'].apply(angle_to_dir)

In [8]:
dangjin_weather_06['Wind_x'] = dangjin_weather_06['Wind_x'].interpolate()
dangjin_weather_06['Wind_y'] = dangjin_weather_06['Wind_y'].interpolate()

ulsan_weather_06['Wind_x'] = ulsan_weather_06['Wind_x'].interpolate()
ulsan_weather_06['Wind_y'] = ulsan_weather_06['Wind_y'].interpolate()

In [9]:
dangjin_weather_06['hour'] = pd.to_datetime(dangjin_weather_06['Forecast_time']).dt.hour
dangjin_weather_06['day'] = pd.to_datetime(dangjin_weather_06['Forecast_time']).dt.day
dangjin_weather_06['month'] = pd.to_datetime(dangjin_weather_06['Forecast_time']).dt.month

ulsan_weather_06['hour'] = pd.to_datetime(ulsan_weather_06['Forecast_time']).dt.hour
ulsan_weather_06['day'] = pd.to_datetime(ulsan_weather_06['Forecast_time']).dt.day
ulsan_weather_06['month'] = pd.to_datetime(ulsan_weather_06['Forecast_time']).dt.month

In [10]:
#이슬점 직접 계산해서 추가
b = 17.62
c = 243.12
gamma = (b * dangjin_weather_06['Temperature'] /(c + dangjin_weather_06['Temperature'])) + np.log(dangjin_weather_06['Humidity'] / 100.0)
dewpoint = (c * gamma) / (b - gamma)
dangjin_weather_06['DewPoint'] = dewpoint  

gamma = (b *ulsan_weather_06['Temperature'] /(c + ulsan_weather_06['Temperature'])) + np.log(ulsan_weather_06['Humidity'] / 100.0)
dewpoint = (c * gamma) / (b - gamma)
ulsan_weather_06['DewPoint'] = dewpoint

In [11]:
dangjin_weather_06['방위각'] = dangjin_weather_06['방위각'].apply(lambda x : int(x[:3]))
dangjin_weather_06['적경'] = dangjin_weather_06['적경'].apply(lambda x : int(x.split(" ")[0]))
dangjin_weather_06['고도'] = dangjin_weather_06['고도'].apply(lambda x : float(x.split(" ")[0] + "." + x.split(" ")[1] + x.split(" ")[2][:2]))
dangjin_weather_06['적위'] = dangjin_weather_06['적위'].apply(lambda x : float(x.split(" ")[0] + "." + x.split(" ")[1] + x.split(" ")[2][:2]))

ulsan_weather_06['방위각'] = ulsan_weather_06['방위각'].apply(lambda x : int(x[:3]))
ulsan_weather_06['고도'] = ulsan_weather_06['고도'].apply(lambda x : int(x.split(" ")[0]))
ulsan_weather_06['적경'] = ulsan_weather_06['적경'].apply(lambda x : float(x.split(" ")[0] + "." + x.split(" ")[1] + x.split(" ")[2][:2]))
ulsan_weather_06['적위'] = ulsan_weather_06['적위'].apply(lambda x : float(x.split(" ")[0] + "." + x.split(" ")[1] + x.split(" ")[2][:2]))

In [12]:
using_col = ['Temperature', 'DewPoint', 'Humidity', 'Cloud', 'Wind_x', 'Wind_y',
             'hour', 'month', '방위각',	'고도',	'적경',	'적위','일조','일사']

dangjin_weather_06 = dangjin_weather_06[using_col].fillna(0)
ulsan_weather_06 = ulsan_weather_06[using_col].fillna(0)

In [14]:
# Shift => period만큼 데이터 옮겨서 concat
# 해당 시간대에 대해 당일 예보, 하루 전 예보, 이틀 전 예보 이렇게 column을 늘려줌
def shift(df, period):
  target = pd.DataFrame()
  for i in range(period, -1, -1): #period부터 거꾸로 1칸씩 (71부터 0까지 출력)
    tmp = df.shift(i) #i는 period
    target = pd.concat([target, tmp], axis = 1)
  return target

In [13]:
dangjin_weather_06

,Temperature,DewPoint,Humidity,Cloud,Wind_x,Wind_y,hour,month,방위각,고도,적경,적위,일조,일사
0,20.2,18.857262,92.0,5.0,9.192533e-01,0.771345,1,6,6,-29.2304,6,23.1914,0.000000,0.000000
1,19.8,18.974882,95.0,5.0,1.836970e-17,0.300000,2,6,21,-26.3224,6,23.1908,0.000000,0.000000
2,19.4,18.408318,94.0,6.0,8.426489e-01,0.707066,3,6,35,-20.5220,6,23.1902,0.000000,0.000000
3,19.7,18.362344,92.0,2.0,3.762222e-01,1.033662,4,6,46,-13.0058,6,23.1856,0.000000,0.000000
4,19.1,18.279263,95.0,6.0,3.078181e-01,0.845723,5,6,57,-3.3515,6,23.1851,0.000000,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
210,25.0,20.257642,75.0,1.0,-2.558775e+00,0.622185,19,7,291,9.2639,7,22.4258,0.861254,0.182235
211,24.0,20.334196,80.0,1.0,-2.131525e+00,0.388645,20,7,300,-1.2008,7,22.4243,0.304918,0.000000
212,23.0,20.340073,85.0,1.0,-1.687328e+00,0.207178,21,7,309,-11.0839,7,22.4228,0.000000,0.000000
213,23.0,20.340073,85.0,1.0,-1.687328e+00,0.207178,22,7,321,-19.3136,7,22.4212,0.000000,0.000000


In [15]:
dangjin_weather_06 = shift(dangjin_weather_06, 71) 
ulsan_weather_06 = shift(ulsan_weather_06, 71)     

In [16]:
dangjin_weather_06

,Temperature,DewPoint,Humidity,Cloud,Wind_x,Wind_y,hour,month,방위각,고도,적경,적위,일조,일사,Temperature,DewPoint,Humidity,Cloud,Wind_x,Wind_y,hour,month,방위각,고도,적경,적위,일조,일사,Temperature,DewPoint,Humidity,Cloud,Wind_x,Wind_y,hour,month,방위각,고도,적경,적위,...,Humidity,Cloud,Wind_x,Wind_y,hour,month,방위각,고도,적경,적위,일조,일사,Temperature,DewPoint,Humidity,Cloud,Wind_x,Wind_y,hour,month,방위각,고도,적경,적위,일조,일사,Temperature,DewPoint,Humidity,Cloud,Wind_x,Wind_y,hour,month,방위각,고도,적경,적위,일조,일사
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,20.2,18.857262,92.0,5.0,9.192533e-01,0.771345,1,6,6,-29.2304,6,23.1914,0.000000,0.000000
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,20.2,18.857262,92.0,5.0,9.192533e-01,0.771345,1.0,6.0,6.0,-29.2304,6.0,23.1914,0.000000,0.000000,19.8,18.974882,95.0,5.0,1.836970e-17,0.300000,2,6,21,-26.3224,6,23.1908,0.000000,0.000000
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,92.000000,5.000000,9.192533e-01,0.771345,1.0,6.0,6.0,-29.2304,6.0,23.1914,0.000000,0.000000,19.8,18.974882,95.0,5.0,1.836970e-17,0.300000,2.0,6.0,21.0,-26.3224,6.0,23.1908,0.000000,0.000000,19.4,18.408318,94.0,6.0,8.426489e-01,0.707066,3,6,35,-20.5220,6,23.1902,0.000000,0.000000
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,95.000000,5.000000,1.836970e-17,0.300000,2.0,6.0,21.0,-26.3224,6.0,23.1908,0.000000,0.000000,19.4,18.408318,94.0,6.0,8.426489e-01,0.707066,3.0,6.0,35.0,-20.5220,6.0,23.1902,0.000000,0.000000,19.7,18.362344,92.0,2.0,3.762222e-01,1.033662,4,6,46,-13.0058,6,23.1856,0.000000,0.000000
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,94.000000,6.000000,8.426489e-01,0.707066,3.0,6.0,35.0,-20.5220,6.0,23.1902,0.000000,0.000000,19.7,18.362344,92.0,2.0,3.762222e-01,1.033662,4.0,6.0,46.0,-13.0058,6.0,23.1856,0.000000,0.000000,19.1,18.279263,95.0,6.0,3.078181e-01,0.845723,5,6,57,-3.3515,6,23.1851,0.000000,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
210,27.8,17.929917,55.0,1.0,0.845723,3.078181e-01,20.0,7.0,300.0,-1.1500,6.0,22.5841,0.4,0.11,25.9,18.565805,64.0,2.0,9.192533e-01,7.713451e-01,21.0,7.0,310.0,-11.0130,6.0,22.5828,0.0,0.0,25.4,18.824371,67.0,5.0,9.396926e-01,3.420201e-01,22.0,7.0,321.0,-19.2200,6.0,22.5816,...,68.333333,1.666667,-2.808749e+00,1.231006,17.0,7.0,275.0,32.4011,6.0,22.4329,0.759280,0.367935,26.0,20.101875,70.0,1.0,-2.964545e+00,0.906352,18.0,7.0,283.0,20.5150,6.0,22.4313,0.890149,0.315362,25.0,20.257642,75.0,1.0,-2.558775e+00,0.622185,19,7,291,9.2639,7,22.4258,0.861254,0.182235
211,25.9,18.565805,64.0,2.0,0.919253,7.713451e-01,21.0,7.0,310.0,-11.0130,6.0,22.5828,0.0,0.00,25.4,18.824371,67.0,5.0,9.396926e-01,3.420201e-01,22.0,7.0,321.0,-19.2200,6.0,22.5816,0.0,0.0,25.5,18.919532,67.0,4.0,1.900000e+00,2.326829e-16,23.0,7.0,334.0,-25.4056,6.0,22.5804,...,70.000000,1.000000,-2.964545e+00,0.906352,18.0,7.0,283.0,20.5150,6.0,22.4313,0.890149,0.315362,25.0,20.257642,75.0,1.0,-2.558775e+00,0.622185,19.0,7.0,291.0,9.2639,7.0,22.4258,0.861254,0.182235,24.0,20.334196,80.0,1.0,-2.131525e+00,0.388645,20,7,300,-1.2008,7,22.4243,0.30

In [17]:
#앞에 72개는 nan 제거

dangjin_weather_06 = dangjin_weather_06[(dangjin_weather_06.isnull().sum(axis = 1) <= 0).values]
ulsan_weather_06 = ulsan_weather_06[(ulsan_weather_06.isnull().sum(axis = 1) <= 0).values]

In [18]:
dangjin_weather_06

,Temperature,DewPoint,Humidity,Cloud,Wind_x,Wind_y,hour,month,방위각,고도,적경,적위,일조,일사,Temperature,DewPoint,Humidity,Cloud,Wind_x,Wind_y,hour,month,방위각,고도,적경,적위,일조,일사,Temperature,DewPoint,Humidity,Cloud,Wind_x,Wind_y,hour,month,방위각,고도,적경,적위,...,Humidity,Cloud,Wind_x,Wind_y,hour,month,방위각,고도,적경,적위,일조,일사,Temperature,DewPoint,Humidity,Cloud,Wind_x,Wind_y,hour,month,방위각,고도,적경,적위,일조,일사,Temperature,DewPoint,Humidity,Cloud,Wind_x,Wind_y,hour,month,방위각,고도,적경,적위,일조,일사
71,20.2,18.857262,92.0,5.0,9.192533e-01,7.713451e-01,1.0,6.0,6.0,-29.2304,6.0,23.1914,0.0,0.00,19.8,18.974882,95.0,5.0,1.836970e-17,3.000000e-01,2.0,6.0,21.0,-26.3224,6.0,23.1908,0.0,0.00,19.4,18.408318,94.0,6.0,8.426489e-01,7.070664e-01,3.0,6.0,35.0,-20.5220,6.0,23.1902,...,89.000000,9.000000,1.224647e-17,2.000000e-01,22.0,6.0,321.0,-19.1545,6.0,23.1043,0.000000,0.000000,21.5,19.967301,91.0,8.0,8.000000e-01,9.797174e-17,23.0,6.0,335.0,-25.3212,6.0,23.1034,0.000000,0.000000,21.1,19.748092,92.0,9.0,1.710101e-01,0.469846,0,6,350,-29.0740,6,23.1025,0.000000,0.000000
72,19.8,18.974882,95.0,5.0,1.836970e-17,3.000000e-01,2.0,6.0,21.0,-26.3224,6.0,23.1908,0.0,0.00,19.4,18.408318,94.0,6.0,8.426489e-01,7.070664e-01,3.0,6.0,35.0,-20.5220,6.0,23.1902,0.0,0.00,19.7,18.362344,92.0,2.0,3.762222e-01,1.033662e+00,4.0,6.0,46.0,-13.0058,6.0,23.1856,...,91.000000,8.000000,8.000000e-01,9.797174e-17,23.0,6.0,335.0,-25.3212,6.0,23.1034,0.000000,0.000000,21.1,19.748092,92.0,9.0,1.710101e-01,4.698463e-01,0.0,6.0,350.0,-29.0740,6.0,23.1025,0.000000,0.000000,20.9,20.237402,96.0,9.0,6.123234e-18,0.100000,1,6,5,-29.3245,6,23.1016,0.000000,0.000000
73,19.4,18.408318,94.0,6.0,8.426489e-01,7.070664e-01,3.0,6.0,35.0,-20.5220,6.0,23.1902,0.0,0.00,19.7,18.362344,92.0,2.0,3.762222e-01,1.033662e+00,4.0,6.0,46.0,-13.0058,6.0,23.1856,0.0,0.00,19.1,18.279263,95.0,6.0,3.078181e-01,8.457234e-01,5.0,6.0,57.0,-3.3515,6.0,23.1851,...,92.000000,9.000000,1.710101e-01,4.698463e-01,0.0,6.0,350.0,-29.0740,6.0,23.1025,0.000000,0.000000,20.9,20.237402,96.0,9.0,6.123234e-18,1.000000e-01,1.0,6.0,5.0,-29.3245,6.0,23.1016,0.000000,0.000000,20.3,19.640407,96.0,9.0,1.224647e-17,0.200000,2,6,21,-26.4335,6,23.1007,0.000000,0.000000
74,19.7,18.362344,92.0,2.0,3.762222e-01,1.033662e+00,4.0,6.0,46.0,-13.0058,6.0,23.1856,0.0,0.00,19.1,18.279263,95.0,6.0,3.078181e-01,8.457234e-01,5.0,6.0,57.0,-3.3515,6.0,23.1851,0.0,0.00,19.7,18.535010,93.0,6.0,8.426489e-01,7.070664e-01,6.0,6.0,65.0,6.5535,6.0,23.1845,...,96.000000,9.000000,6.123234e-18,1.000000e-01,1.0,6.0,5.0,-29.3245,6.0,23.1016,0.000000,0.000000,20.3,19.640407,96.0,9.0,1.224647e-17,2.000000e-01,2.0,6.0,21.0,-26.4335,6.0,23.1007,0.000000,0.000000,20.3,19.807533,97.0,10.0,6.123234e-18,0.100000,3,6,34,-21.0422,6,23.0959,0.000000,0.000000
75,19.1,18.279263,95.0,6.0,3.078181e-01,8.457234e-01,5.0,6.0,57.0,-3.3515,6.0,23.1851,0.0,0.00,19.7,18.535010,93.0,6.0,8.426489e-01,7.070664e-01,6.0,6.0,65.0,6.5535,6.0,23.1845,0.1,0.06,21.4,18.582460,84.0,4.0,1.455484e+00,1.221296e+00,7.0,6.0,74.0,18.1003,6.0,23.1839,...,96.000000,9.000000,1.224647e-17,2.000000e-01,2.0,6.0,21.0,-26.4335,6.0,23.1007,0.000000,0.000000,20.3,19.807533,97.0,10.0,6.123234e-18,1.000000e-01,3.0,6.0,34.0,-21.0422,6.0,23.0959,0.000000,0.000000,20.2,19.873403,98.0,10.0,6.123234e-18,0.100000,4,6,46,-13.1320,6,23.0950,0.000000,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
210,27.8,17.929917,55.0,1.0,8.457234e-01,3.078181e-01,20.0,7.0,300.0,-1.1500,6.0,22.5841,0.4,0.11,25.9,18.565805,64.0,2.0,9.192533e-01,7.713451e-01,21.0,7.0,310.0,-11.0130,6.0,22.5828,0.0,0.00,25.4,18.824371,67.0,5.0,9.396926e-01,3.420201e-01,22.0,7.0,321.0,-19.2200,6.0,22.5816,...,68.333333,1.666667,-2.808749e+00,1.231006e+00,17.0,7.0,275.0,32.4011,6.0,22.4329,0.759280,0.367935,26

In [19]:
#min max scaling
dangjin_min = dangjin_weather_06.min()
dangjin_max = dangjin_weather_06.max()

dangjin_weather_06 = (dangjin_weather_06 - dangjin_min) / (dangjin_max - dangjin_min)

In [20]:
ulsan_min = ulsan_weather_06.min()
ulsan_max = ulsan_weather_06.max()

ulsan_weather_06 = (ulsan_weather_06 - ulsan_min) / (ulsan_max - ulsan_min)

# 2D Conv 모델

In [21]:
import tensorflow as tf

In [22]:
dangjin_weather_06.shape

(144, 1008)

In [23]:
dangjin_weather_06 = dangjin_weather_06.values.reshape((dangjin_weather_06.shape[0], 72, 14, 1)) #(193, 864) -> (193, 72, 12, 1)
ulsan_weather_06 = ulsan_weather_06.values.reshape((ulsan_weather_06.shape[0], 72, 14, 1))

In [24]:
dangjin_weather_06.shape

(144, 72, 14, 1)

In [ ]:
'''
dangjin_weather_06 = tf.image.resize(dangjin_weather_06, (72, 32)) 
ulsan_weather_06 = tf.image.resize(ulsan_weather_06, (72, 32))
'''

'\ndangjin_weather_06 = tf.image.resize(dangjin_weather_06, (72, 32)) \nulsan_weather_06 = tf.image.resize(ulsan_weather_06, (72, 32))\n'

In [25]:
dangjin_weather_06 = np.squeeze(dangjin_weather_06, axis = -1) 
dangjin_weather_06 = np.stack((dangjin_weather_06,)*3, axis = -1)  

ulsan_weather_06 = np.squeeze(ulsan_weather_06, axis = -1)
ulsan_weather_06 = np.stack((ulsan_weather_06,)*3, axis = -1)

In [26]:
ulsan_weather_06.shape

(144, 72, 14, 3)

In [27]:
dangjin_test = dangjin_weather_06[-24:]
ulsan_test = ulsan_weather_06[-24:]

In [28]:
dangjin_test.shape

(24, 72, 14, 3)

* Inference

In [29]:
floating_model0 = tf.keras.models.load_model("/content/gdrive/MyDrive/dacon_timeseries/final_model/model/floating_fold0.h5")
floating_model1 = tf.keras.models.load_model("/content/gdrive/MyDrive/dacon_timeseries/final_model/model/floating_fold1.h5")
floating_model2 = tf.keras.models.load_model("/content/gdrive/MyDrive/dacon_timeseries/final_model/model/floating_fold2.h5")
floating_model3 = tf.keras.models.load_model("/content/gdrive/MyDrive/dacon_timeseries/final_model/model/floating_fold3.h5")
floating_model4 = tf.keras.models.load_model("/content/gdrive/MyDrive/dacon_timeseries/final_model/model/floating_fold4.h5")

In [30]:
floating0 = floating_model0.predict(dangjin_test)
floating1 = floating_model1.predict(dangjin_test)
floating2 = floating_model2.predict(dangjin_test)
floating3 = floating_model3.predict(dangjin_test)
floating4 = floating_model4.predict(dangjin_test)

In [31]:
floating2

array([[  2.533438 ],
       [  2.3309877],
       [  2.1450984],
       [  2.15785  ],
       [  2.3264709],
       [  2.5962758],
       [  3.044877 ],
       [  5.579776 ],
       [ 15.112176 ],
       [ 81.66041  ],
       [175.76793  ],
       [230.17918  ],
       [284.40634  ],
       [270.6836   ],
       [221.0266   ],
       [184.65211  ],
       [155.54114  ],
       [128.2817   ],
       [ 93.060165 ],
       [ 26.988052 ],
       [  3.3330204],
       [  1.7536919],
       [  0.9345366],
       [  1.0722908]], dtype=float32)

In [32]:
warehouse_model0 = tf.keras.models.load_model("/content/gdrive/MyDrive/dacon_timeseries/final_model/model/warehouse_fold0.h5")
warehouse_model1 = tf.keras.models.load_model("/content/gdrive/MyDrive/dacon_timeseries/final_model/model/warehouse_fold1.h5")
warehouse_model2 = tf.keras.models.load_model("/content/gdrive/MyDrive/dacon_timeseries/final_model/model/warehouse_fold2.h5")
warehouse_model3 = tf.keras.models.load_model("/content/gdrive/MyDrive/dacon_timeseries/final_model/model/warehouse_fold3.h5")
warehouse_model4 = tf.keras.models.load_model("/content/gdrive/MyDrive/dacon_timeseries/final_model/model/warehouse_fold4.h5")

In [33]:
warehouse0 = warehouse_model0.predict(dangjin_test)
warehouse1 = warehouse_model1.predict(dangjin_test)
warehouse2 = warehouse_model2.predict(dangjin_test)
warehouse3 = warehouse_model3.predict(dangjin_test)
warehouse4 = warehouse_model4.predict(dangjin_test)

In [34]:
dangjin_model0 = tf.keras.models.load_model("/content/gdrive/MyDrive/dacon_timeseries/final_model/model/dangjin_fold0.h5")
dangjin_model1 = tf.keras.models.load_model("/content/gdrive/MyDrive/dacon_timeseries/final_model/model/dangjin_fold1.h5")
dangjin_model2 = tf.keras.models.load_model("/content/gdrive/MyDrive/dacon_timeseries/final_model/model/dangjin_fold2.h5")
dangjin_model3 = tf.keras.models.load_model("/content/gdrive/MyDrive/dacon_timeseries/final_model/model/dangjin_fold3.h5")
dangjin_model4 = tf.keras.models.load_model("/content/gdrive/MyDrive/dacon_timeseries/final_model/model/dangjin_fold4.h5")

In [35]:
dangjin0 = dangjin_model0.predict(dangjin_test)
dangjin1 = dangjin_model1.predict(dangjin_test)
dangjin2 = dangjin_model2.predict(dangjin_test)
dangjin3 = dangjin_model3.predict(dangjin_test)
dangjin4 = dangjin_model4.predict(dangjin_test)

In [36]:
ulsan_model0 = tf.keras.models.load_model("/content/gdrive/MyDrive/dacon_timeseries/final_model/model/ulsan_fold0.h5")
ulsan_model1 = tf.keras.models.load_model("/content/gdrive/MyDrive/dacon_timeseries/final_model/model/ulsan_fold1.h5")
ulsan_model2 = tf.keras.models.load_model("/content/gdrive/MyDrive/dacon_timeseries/final_model/model/ulsan_fold2.h5")
ulsan_model3 = tf.keras.models.load_model("/content/gdrive/MyDrive/dacon_timeseries/final_model/model/ulsan_fold3.h5")
ulsan_model4 = tf.keras.models.load_model("/content/gdrive/MyDrive/dacon_timeseries/final_model/model/ulsan_fold4.h5")

In [37]:
dangjin_test.shape

(24, 72, 14, 3)

In [38]:
ulsan_test.shape

(24, 72, 14, 3)

In [39]:
ulsan0 = ulsan_model0.predict(ulsan_test)
ulsan1 = ulsan_model1.predict(ulsan_test)
ulsan2 = ulsan_model2.predict(ulsan_test)
ulsan3 = ulsan_model3.predict(ulsan_test)
ulsan4 = ulsan_model4.predict(ulsan_test)

In [40]:

floating = (floating0 + floating1 + floating2 + floating3 + floating4) / 5
warehouse = (warehouse0 + warehouse1 + warehouse2 + warehouse3 + warehouse4 ) / 5
dangjin = (dangjin0 + dangjin1 + dangjin2 + dangjin3 + dangjin4) / 5
ulsan = (ulsan0 + ulsan1 + ulsan2 + ulsan3 + ulsan4) / 5


In [ ]:
'''
floating = (floating0 + floating1 + floating2) / 3
warehouse = (warehouse0 + warehouse1 + warehouse2 ) / 3
dangjin = (dangjin0 + dangjin1 + dangjin2) / 3
ulsan = (ulsan0 + ulsan1 + ulsan2 ) / 3
'''

In [41]:
#sample_submission2 = pd.read_csv("/content/gdrive/MyDrive/dacon_timeseries/sample_submission.csv")
sample_submission = pd.read_csv("/content/gdrive/MyDrive/dacon_timeseries/final_model/private_data/neural_prophet_validation.csv")

In [ ]:
#sample_submission2

In [42]:
sample_submission

,time,dangjin_floating,dangjin_warehouse,dangjin,ulsan
0,2021-02-01 01:00:00,0.0,0.000000,0.0,0.000000
1,2021-02-01 02:00:00,0.0,0.000000,0.0,0.000000
2,2021-02-01 03:00:00,0.0,0.000000,0.0,0.000000
3,2021-02-01 04:00:00,0.0,0.000000,0.0,0.000000
4,2021-02-01 05:00:00,0.0,0.000000,0.0,0.651203
...,...,...,...,...,...
1387,2021-07-08 20:00:00,0.0,1.125614,0.0,2.495743
1388,2021-07-08 21:00:00,0.0,1.879234,0.0,3.404133
1389,2021-07-08 22:00:00,0.0,0.933403,0.0,4.150723
1390,2021-07-08 23:00:00,0.0,0.127186,0.0,3.220158


* 시간은 내일 00시부터 23시까지 !! (가져온 날씨 데이터가 그럼)

In [48]:
#loc는 끝 인덱스도 포함....
#sample_submission.loc[815:839] #6/14일 제출
#sample_submission.loc[839:862] #6/15일
#sample_submission.loc[863:886] #6/16일
#sample_submission.loc[887:910] #6/17일
#sample_submission.loc[911:934] #6/18일
#sample_submission.loc[935:958] #6/19일
#sample_submission.loc[959:982] #6/20일
#sample_submission.loc[983:1006] #6/20일
sample_submission.loc[1295:1318] #7/4

,time,dangjin_floating,dangjin_warehouse,dangjin,ulsan
1295,2021-07-04 24:00:00,0.000000,9.842110,0.000000,2.825092
1296,2021-07-05 01:00:00,0.000000,11.000393,0.000000,3.311150
1297,2021-07-05 02:00:00,0.000000,10.555397,0.000000,2.956802
1298,2021-07-05 03:00:00,0.000000,11.174835,0.000000,2.060638
1299,2021-07-05 04:00:00,0.000000,12.575165,0.143097,2.994717
1300,2021-07-05 05:00:00,0.000000,11.294235,2.055847,4.153015
1301,2021-07-05 06:00:00,0.000000,8.524754,0.000000,2.686081
1302,2021-07-05 07:00:00,0.000000,13.815281,0.000000,6.151508
1303,2021-07-05 08:00:00,26.695946,40.630936,35.643776,30.926323
1304,2021-07-05 09:00:00,94.792450,97.584351,121.196243,81.947777


In [49]:
floating

array([[  2.4601412],
       [  2.5823867],
       [  2.5504615],
       [  2.43695  ],
       [  2.444633 ],
       [  2.4066174],
       [  2.8081899],
       [  5.6439066],
       [ 20.768415 ],
       [ 94.70459  ],
       [198.16928  ],
       [270.14468  ],
       [327.27716  ],
       [312.74536  ],
       [273.16754  ],
       [229.05562  ],
       [167.22116  ],
       [110.38139  ],
       [ 63.569733 ],
       [ 14.143152 ],
       [  2.0237184],
       [  1.4589626],
       [  1.5775397],
       [  1.883469 ]], dtype=float32)

In [50]:
warehouse

array([[  1.8233902],
       [  1.7961569],
       [  1.9179659],
       [  2.1867037],
       [  2.3338091],
       [  2.3344638],
       [  2.3360944],
       [  2.5504143],
       [  7.017029 ],
       [ 32.971817 ],
       [ 77.30202  ],
       [113.95321  ],
       [148.9844   ],
       [146.87657  ],
       [128.82712  ],
       [106.25604  ],
       [ 81.41909  ],
       [ 48.654377 ],
       [ 19.418142 ],
       [  2.9357414],
       [  1.699897 ],
       [  1.7070005],
       [  1.7396553],
       [  1.9013088]], dtype=float32)

In [ ]:
floating.shape

(24, 1)

In [ ]:
sample_submission.loc[1295:1318].shape

(24, 5)

In [51]:
sample_submission.loc[1295:1318, 'dangjin_floating'] = floating
sample_submission.loc[1295:1318, 'dangjin_warehouse'] = warehouse
sample_submission.loc[1295:1318, 'dangjin'] = dangjin
sample_submission.loc[1295:1318, 'ulsan'] = ulsan

In [52]:
sample_submission.loc[1295:1318]

,time,dangjin_floating,dangjin_warehouse,dangjin,ulsan
1295,2021-07-04 24:00:00,2.460141,1.823390,5.010951,126.312035
1296,2021-07-05 01:00:00,2.582387,1.796157,5.071882,126.312035
1297,2021-07-05 02:00:00,2.550462,1.917966,5.074673,126.312035
1298,2021-07-05 03:00:00,2.436950,2.186704,4.664258,126.312035
1299,2021-07-05 04:00:00,2.444633,2.333809,4.420192,126.312035
1300,2021-07-05 05:00:00,2.406617,2.334464,4.881239,126.312035
1301,2021-07-05 06:00:00,2.808190,2.336094,4.874202,126.312035
1302,2021-07-05 07:00:00,5.643907,2.550414,5.437247,126.312035
1303,2021-07-05 08:00:00,20.768415,7.017029,14.041563,126.312035
1304,2021-07-05 09:00:00,94.704590,32.971817,86.920578,126.312035


In [53]:
sample_submission.to_csv("/content/gdrive/MyDrive/dacon_timeseries/final_model/0704_submission.csv", index = False)

In [ ]:
ls